In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import trimesh
import argparse
import os
import time
import numpy as np
import torch
import torch.optim as optim
import lib.utils as utils
from lib.utils import standard_normal_logprob
from lib.utils import count_nfe, count_total_time
from lib.utils import build_model_tabular
import pandas as pd
import random
from lib.visualize_flow import visualize_transform
import lib.layers.odefunc as odefunc
from matplotlib import pyplot as plt
%matplotlib inline 
import glob

In [2]:
SOLVERS = ["dopri5"]
parser = argparse.ArgumentParser('SoftFlow')
parser.add_argument(
    '--data', choices=['2spirals_1d','2spirals_2d', 'swissroll_1d','swissroll_2d', 'circles_1d', 'circles_2d', '2sines_1d', 'target_1d'],
    type=str, default='3d_microgel'
)
parser.add_argument("--layer_type", type=str, default="concatsquash", choices=["concatsquash"])
parser.add_argument('--dims', type=str, default='64-64-64')
parser.add_argument("--num_blocks", type=int, default=1, help='Number of stacked CNFs.')
parser.add_argument('--time_length', type=float, default=0.5)
parser.add_argument('--train_T', type=eval, default=True)
parser.add_argument("--divergence_fn", type=str, default="brute_force", choices=["brute_force", "approximate"])
parser.add_argument("--nonlinearity", type=str, default="tanh", choices=odefunc.NONLINEARITIES)

parser.add_argument('--solver', type=str, default='dopri5', choices=SOLVERS)
parser.add_argument('--atol', type=float, default=1e-5)
parser.add_argument('--rtol', type=float, default=1e-5)

parser.add_argument('--residual', type=eval, default=False, choices=[True, False])
parser.add_argument('--rademacher', type=eval, default=False, choices=[True, False])
parser.add_argument('--spectral_norm', type=eval, default=False, choices=[True, False])
parser.add_argument('--niters', type=int, default=100000)
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--test_batch_size', type=int, default=1000)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', type=float, default=1e-5)

# for the proposed method
parser.add_argument('--std_min', type=float, default=0.0)
parser.add_argument('--std_max', type=float, default=0.1)
parser.add_argument('--std_weight', type=float, default=2)

parser.add_argument('--viz_freq', type=int, default=100)
parser.add_argument('--val_freq', type=int, default=400)
parser.add_argument('--log_freq', type=int, default=10)
parser.add_argument('--gpu', type=int, default=0)
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
# logger
args.data = "microgel_multiclass"
save_path = './results/' + args.data + '/SoftFlow'
utils.makedirs(save_path)

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [4]:
def get_transforms(model):

    def sample_fn(z, logpz=None):
        zeors_std = torch.zeros(z.shape[0], 1).to(z)
        if logpz is not None:
            return model(z, zeors_std, logpz, reverse=True)
        else:
            return model(z, zeors_std, reverse=True)

    def density_fn(x, logpx=None):
        zeors_std = torch.zeros(x.shape[0], 1).to(x)
        if logpx is not None:
            return model(x, zeors_std, logpx, reverse=False)
        else:
            return model(x, zeors_std, reverse=False)

    return sample_fn, density_fn

DATA_add = "/home/staff/azad/deeplearning/pointcloud/PAINT_DiffTemp/Core-shell/"
Type     = ['Microgel_plotter_v1_density', 'Microgel_plotter_v2_solvatochromism']

temperature_list = os.listdir(DATA_add)
temperature_list.sort()
temperature_list.remove('35C')

def get_data():
    idt = random.randint(0,len(temperature_list)-1)
    DATA_list = glob.glob(DATA_add+ temperature_list[idt]+'/'+Type[0]+"/*.3d")
    temperature = int(temperature_list[0][-3:-1])
    indx = random.randint(0,len(DATA_list)-1)
    DATA = pd.read_csv(DATA_list[indx], delimiter = "\t", header=None)
    DATA = DATA.to_numpy()[:,:3]
    DATA -= np.mean(DATA, axis=0) 
    DATA = (DATA-np.min(DATA, axis=0))/(np.max(DATA, axis=0)-np.min(DATA, axis=0)) 
    newrow = [temperature, temperature, temperature]
    DATA = np.vstack([newrow, DATA])

    return DATA



def compute_loss(args, model, batch_size=None):
    if batch_size is None: batch_size = args.batch_size
    x = get_data()


    x = torch.from_numpy(x).type(torch.float32).to(device)
    zero = torch.zeros(x.shape[0], 1).to(x)

    # transform to z
    std = (args.std_max - args.std_min) * torch.rand_like(x[:,0]).view(-1,1) + args.std_min
    eps = torch.randn_like(x) * std
    std_in = std / args.std_max * args.std_weight
#     print(x.shape, std_in.shape, zero.shape)
    z, delta_logp = model(x+eps, std_in, zero)

    # compute log q(z)
    logpz = standard_normal_logprob(z).sum(1, keepdim=True)

    logpx = logpz - delta_logp
    loss = -torch.mean(logpx)
    return loss

In [5]:
model = build_model_tabular(args, 3).to(device)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

time_meter = utils.RunningAverageMeter(0.93)
loss_meter = utils.RunningAverageMeter(0.93)
nfef_meter = utils.RunningAverageMeter(0.93)
nfeb_meter = utils.RunningAverageMeter(0.93)
tt_meter = utils.RunningAverageMeter(0.93)

end = time.time()
best_loss = float('inf')

In [6]:
LOW = -4
HIGH = 4
def visualize3d(samples, prior_sample, prior_logdensity, n_samples, transform, inverse_transform, memory=100, title1="$x ~ p(x)$", title2="$q(x)$", title3="$x ~ q(x)$", device="cpu", npts = 100):
    ## first one
    fig = plt.figure(figsize=(25,5))
    ax1 = fig.add_subplot(1, 3, 1, projection="3d")
    ax1.scatter(samples[:, 0], samples[:, 1], samples[:, 2], s=1, c='b', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title1)
    
    ## second one
    side = np.linspace(LOW, HIGH, npts)
    xx, yy, zz = np.meshgrid(side, side, side)
    memory=100
    x = np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1), zz.reshape(-1, 1)])
    x = torch.from_numpy(x).type(torch.float32).to(device)
    zeros = torch.zeros(x.shape[0], 1).to(x)

    z, delta_logp = [], []
    inds = torch.arange(0, x.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        z_, delta_logp_ = inverse_transform(x[ii], zeros[ii])
        z.append(z_)
        delta_logp.append(delta_logp_)
    z = torch.cat(z, 0)
    delta_logp = torch.cat(delta_logp, 0)

    logpz = prior_logdensity(z).view(z.shape[0], -1).sum(1, keepdim=True)  # logp(z)
    logpx = logpz - delta_logp

    px = np.exp(logpx.detach().cpu().numpy()).reshape(npts, npts,npts)
    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(px[:,:,1])
    ax.set_title(title2)
    
    ## Third one
    z = prior_sample(n_samples, 3).type(torch.float32).to(device)
    zk = []
    inds = torch.arange(0, z.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        zk.append(transform(z[ii]))

    zk = torch.cat(zk, 0).detach().cpu().numpy()

    ax1 = fig.add_subplot(1, 3, 3, projection="3d")
    ax1.scatter(zk[:, 0], zk[:, 1], zk[:, 2], s=1, c='r', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title3)


In [ ]:
model.train()
for itr in range(1, args.niters + 1):
    optimizer.zero_grad()

    loss = compute_loss(args, model)
    

    loss_meter.update(loss.item())

    total_time = count_total_time(model)
    nfe_forward = count_nfe(model)

    loss.backward()
    optimizer.step()

    nfe_total = count_nfe(model)
    nfe_backward = nfe_total - nfe_forward
    nfef_meter.update(nfe_forward)
    nfeb_meter.update(nfe_backward)

    time_meter.update(time.time() - end)
    tt_meter.update(total_time)

    log_message = (
        'Iter {:04d} | Time {:.4f}({:.4f}) | Loss {:.6f}({:.6f}) | NFE Forward {:.0f}({:.1f})'
        ' | NFE Backward {:.0f}({:.1f}) | CNF Time {:.4f}({:.4f})'.format(
            itr, time_meter.val, time_meter.avg, loss_meter.val, loss_meter.avg, nfef_meter.val, nfef_meter.avg,
            nfeb_meter.val, nfeb_meter.avg, tt_meter.val, tt_meter.avg
        )
    )

    print(log_message)
    
    if itr % args.val_freq == 0 or itr == args.niters:
        with torch.no_grad():
            model.eval()
            test_loss = compute_loss(args, model, batch_size=args.test_batch_size)
            test_nfe = count_nfe(model)
            log_message = '[TEST] Iter {:04d} | Test Loss {:.6f} | NFE {:.0f}'.format(itr, test_loss, test_nfe)
            print(log_message)

            if test_loss.item() < best_loss:
                best_loss = test_loss.item()
                utils.makedirs(save_path)
                torch.save({
                    'args': args,
                    'state_dict': model.state_dict(),
                }, os.path.join(save_path, 'checkpt.pth'))
            model.train()

    if itr % args.viz_freq == 0:
        with torch.no_grad():
            model.eval()
#             p_samples = toy_data.inf_train_gen(args.data, batch_size=2000)
            p_samples = get_data()

            sample_fn, density_fn = get_transforms(model)
            p_samples = p_samples[1:]
            plt.figure(figsize=(9, 3))
            visualize3d(p_samples, torch.randn, standard_normal_logprob, p_samples.shape[0], sample_fn,density_fn, device=device)
            fig_filename = os.path.join(save_path, 'figs', '{:04d}.jpg'.format(itr))
            utils.makedirs(os.path.dirname(fig_filename))
            plt.savefig(fig_filename, format='png')#, dpi=1200)
            plt.close()
            model.train()

    end = time.time()

print('Training has finished.')

Iter 0001 | Time 0.7457(0.7457) | Loss 3.460724(3.460724) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0002 | Time 0.2229(0.7091) | Loss 3.442682(3.459461) | NFE Forward 26(20.4) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0003 | Time 0.2207(0.6749) | Loss 3.563042(3.466712) | NFE Forward 26(20.8) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0004 | Time 0.1727(0.6397) | Loss 3.287864(3.454193) | NFE Forward 20(20.8) | NFE Backward 20(25.6) | CNF Time 0.5000(0.5000)
Iter 0005 | Time 0.1730(0.6071) | Loss 3.264372(3.440905) | NFE Forward 20(20.7) | NFE Backward 20(25.2) | CNF Time 0.5000(0.5000)
Iter 0006 | Time 0.2223(0.5801) | Loss 3.204510(3.424358) | NFE Forward 26(21.1) | NFE Backward 26(25.2) | CNF Time 0.5000(0.5000)
Iter 0007 | Time 0.2027(0.5537) | Loss 3.349168(3.419094) | NFE Forward 20(21.0) | NFE Backward 26(25.3) | CNF Time 0.5000(0.5000)
Iter 0008 | Time 0.1728(0.5270) | Loss 3.329600(3.412830) | NFE Forward 20(20.9) | 

Iter 0064 | Time 0.2898(0.2671) | Loss 1.938774(2.442336) | NFE Forward 38(32.2) | NFE Backward 32(30.1) | CNF Time 0.5000(0.5000)
Iter 0065 | Time 0.2843(0.2683) | Loss 2.073342(2.416506) | NFE Forward 38(32.6) | NFE Backward 32(30.2) | CNF Time 0.5000(0.5000)
Iter 0066 | Time 0.2899(0.2698) | Loss 1.671354(2.364345) | NFE Forward 38(33.0) | NFE Backward 32(30.3) | CNF Time 0.5000(0.5000)
Iter 0067 | Time 0.2848(0.2709) | Loss 2.002984(2.339050) | NFE Forward 38(33.3) | NFE Backward 32(30.5) | CNF Time 0.5000(0.5000)
Iter 0068 | Time 0.2902(0.2722) | Loss 1.762262(2.298675) | NFE Forward 38(33.6) | NFE Backward 32(30.6) | CNF Time 0.5000(0.5000)
Iter 0069 | Time 0.2890(0.2734) | Loss 1.536389(2.245315) | NFE Forward 38(33.9) | NFE Backward 32(30.7) | CNF Time 0.5000(0.5000)
Iter 0070 | Time 0.2919(0.2747) | Loss 1.620071(2.201548) | NFE Forward 38(34.2) | NFE Backward 32(30.8) | CNF Time 0.5000(0.5000)
Iter 0071 | Time 0.2850(0.2754) | Loss 1.820957(2.174906) | NFE Forward 38(34.5) | 

Iter 0127 | Time 0.3600(0.3686) | Loss 0.619140(0.553743) | NFE Forward 44(43.3) | NFE Backward 44(44.1) | CNF Time 0.5000(0.5000)
Iter 0128 | Time 0.3665(0.3684) | Loss -0.087883(0.508829) | NFE Forward 44(43.4) | NFE Backward 44(44.1) | CNF Time 0.5000(0.5000)
Iter 0129 | Time 0.3656(0.3682) | Loss 0.047504(0.476537) | NFE Forward 44(43.4) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0130 | Time 0.3599(0.3676) | Loss 0.249466(0.460642) | NFE Forward 44(43.4) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0131 | Time 0.3603(0.3671) | Loss 0.702270(0.477556) | NFE Forward 44(43.5) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0132 | Time 0.3673(0.3671) | Loss 0.277961(0.463584) | NFE Forward 44(43.5) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0133 | Time 0.3701(0.3673) | Loss 0.191937(0.444569) | NFE Forward 44(43.6) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0134 | Time 0.3691(0.3675) | Loss 0.265467(0.432032) | NFE Forward 44(43.6) |

Iter 0190 | Time 0.3895(0.3865) | Loss -0.540698(-0.424151) | NFE Forward 50(48.7) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0191 | Time 0.3997(0.3874) | Loss -0.445047(-0.425614) | NFE Forward 50(48.8) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0192 | Time 0.4220(0.3899) | Loss -0.929731(-0.460902) | NFE Forward 50(48.9) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0193 | Time 0.3926(0.3900) | Loss -0.232660(-0.444925) | NFE Forward 50(49.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0194 | Time 0.3908(0.3901) | Loss -1.064582(-0.488301) | NFE Forward 50(49.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0195 | Time 0.4211(0.3923) | Loss -0.824873(-0.511861) | NFE Forward 50(49.1) | NFE Backward 50(44.4) | CNF Time 0.5000(0.5000)
Iter 0196 | Time 0.4415(0.3957) | Loss 0.032769(-0.473737) | NFE Forward 50(49.2) | NFE Backward 56(45.3) | CNF Time 0.5000(0.5000)
Iter 0197 | Time 0.4218(0.3975) | Loss -0.813696(-0.497534) | NFE Forw

Iter 0252 | Time 0.4174(0.4231) | Loss -0.649523(-0.594331) | NFE Forward 50(49.8) | NFE Backward 50(50.7) | CNF Time 0.5000(0.5000)
Iter 0253 | Time 0.4513(0.4250) | Loss -0.406054(-0.581152) | NFE Forward 50(49.8) | NFE Backward 56(51.1) | CNF Time 0.5000(0.5000)
Iter 0254 | Time 0.4206(0.4247) | Loss -0.567021(-0.580163) | NFE Forward 50(49.8) | NFE Backward 50(51.0) | CNF Time 0.5000(0.5000)
Iter 0255 | Time 0.4491(0.4264) | Loss -0.586812(-0.580628) | NFE Forward 50(49.8) | NFE Backward 56(51.4) | CNF Time 0.5000(0.5000)
Iter 0256 | Time 0.4409(0.4274) | Loss -0.474714(-0.573214) | NFE Forward 50(49.9) | NFE Backward 56(51.7) | CNF Time 0.5000(0.5000)
Iter 0257 | Time 0.4505(0.4291) | Loss -0.079049(-0.538622) | NFE Forward 50(49.9) | NFE Backward 56(52.0) | CNF Time 0.5000(0.5000)
Iter 0258 | Time 0.4185(0.4283) | Loss -1.010883(-0.571681) | NFE Forward 50(49.9) | NFE Backward 50(51.8) | CNF Time 0.5000(0.5000)
Iter 0259 | Time 0.4130(0.4272) | Loss -0.487694(-0.565802) | NFE For

...

Iter 1182 | Time 0.4160(0.4179) | Loss -0.568610(-0.762165) | NFE Forward 50(49.7) | NFE Backward 50(49.9) | CNF Time 0.5000(0.5000)
Iter 1183 | Time 0.4447(0.4198) | Loss -0.542809(-0.746810) | NFE Forward 50(49.7) | NFE Backward 56(50.4) | CNF Time 0.5000(0.5000)
Iter 1184 | Time 0.4535(0.4222) | Loss -0.590033(-0.735835) | NFE Forward 50(49.8) | NFE Backward 56(50.8) | CNF Time 0.5000(0.5000)
Iter 1185 | Time 0.4253(0.4224) | Loss -1.149170(-0.764769) | NFE Forward 50(49.8) | NFE Backward 50(50.7) | CNF Time 0.5000(0.5000)
Iter 1186 | Time 0.4172(0.4220) | Loss -0.297066(-0.732030) | NFE Forward 50(49.8) | NFE Backward 50(50.7) | CNF Time 0.5000(0.5000)
Iter 1187 | Time 0.4434(0.4235) | Loss -0.164426(-0.692297) | NFE Forward 50(49.8) | NFE Backward 56(51.0) | CNF Time 0.5000(0.5000)
Iter 1188 | Time 0.4491(0.4253) | Loss -0.032891(-0.646139) | NFE Forward 50(49.8) | NFE Backward 50(51.0) | CNF Time 0.5000(0.5000)
Iter 1189 | Time 0.4198(0.4249) | Loss -1.072665(-0.675996) | NFE For

Iter 1244 | Time 0.3923(0.4172) | Loss -0.738319(-0.669192) | NFE Forward 50(49.8) | NFE Backward 44(49.4) | CNF Time 0.5000(0.5000)
Iter 1245 | Time 0.4216(0.4176) | Loss -0.847260(-0.681657) | NFE Forward 50(49.8) | NFE Backward 50(49.5) | CNF Time 0.5000(0.5000)
Iter 1246 | Time 0.3933(0.4159) | Loss -0.595323(-0.675614) | NFE Forward 50(49.8) | NFE Backward 44(49.1) | CNF Time 0.5000(0.5000)
Iter 1247 | Time 0.4226(0.4163) | Loss -0.453439(-0.660061) | NFE Forward 50(49.8) | NFE Backward 50(49.1) | CNF Time 0.5000(0.5000)
Iter 1248 | Time 0.4203(0.4166) | Loss -1.091645(-0.690272) | NFE Forward 50(49.8) | NFE Backward 50(49.2) | CNF Time 0.5000(0.5000)
Iter 1249 | Time 0.4155(0.4165) | Loss -0.871626(-0.702967) | NFE Forward 50(49.9) | NFE Backward 50(49.3) | CNF Time 0.5000(0.5000)
Iter 1250 | Time 0.4353(0.4178) | Loss -0.237755(-0.670402) | NFE Forward 50(49.9) | NFE Backward 50(49.3) | CNF Time 0.5000(0.5000)
Iter 1251 | Time 0.4727(0.4217) | Loss -0.080226(-0.629090) | NFE For

Iter 1306 | Time 0.4199(0.4109) | Loss -1.249917(-0.740584) | NFE Forward 50(49.8) | NFE Backward 50(48.3) | CNF Time 0.5000(0.5000)
Iter 1307 | Time 0.4058(0.4106) | Loss -1.226392(-0.774590) | NFE Forward 44(49.4) | NFE Backward 50(48.4) | CNF Time 0.5000(0.5000)
Iter 1308 | Time 0.4222(0.4114) | Loss -0.480462(-0.754001) | NFE Forward 50(49.4) | NFE Backward 50(48.5) | CNF Time 0.5000(0.5000)
Iter 1309 | Time 0.4228(0.4122) | Loss -0.358436(-0.726312) | NFE Forward 50(49.4) | NFE Backward 50(48.6) | CNF Time 0.5000(0.5000)
Iter 1310 | Time 0.4233(0.4130) | Loss -0.533360(-0.712805) | NFE Forward 50(49.5) | NFE Backward 50(48.7) | CNF Time 0.5000(0.5000)
Iter 1311 | Time 0.4203(0.4135) | Loss -0.624980(-0.706657) | NFE Forward 50(49.5) | NFE Backward 50(48.8) | CNF Time 0.5000(0.5000)
Iter 1312 | Time 0.4521(0.4162) | Loss -0.285781(-0.677196) | NFE Forward 50(49.6) | NFE Backward 56(49.3) | CNF Time 0.5000(0.5000)
Iter 1313 | Time 0.4217(0.4166) | Loss -0.647350(-0.675107) | NFE For

Iter 1368 | Time 0.4254(0.4177) | Loss -0.569967(-0.616082) | NFE Forward 50(49.8) | NFE Backward 50(49.2) | CNF Time 0.5000(0.5000)
Iter 1369 | Time 0.4220(0.4180) | Loss -0.435747(-0.603458) | NFE Forward 50(49.8) | NFE Backward 50(49.3) | CNF Time 0.5000(0.5000)
Iter 1370 | Time 0.4216(0.4182) | Loss -0.706295(-0.610657) | NFE Forward 50(49.8) | NFE Backward 50(49.3) | CNF Time 0.5000(0.5000)
Iter 1371 | Time 0.3938(0.4165) | Loss -0.923919(-0.632585) | NFE Forward 50(49.8) | NFE Backward 44(49.0) | CNF Time 0.5000(0.5000)
Iter 1372 | Time 0.4239(0.4170) | Loss -0.726046(-0.639127) | NFE Forward 50(49.8) | NFE Backward 50(49.0) | CNF Time 0.5000(0.5000)
Iter 1373 | Time 0.4158(0.4170) | Loss -0.196029(-0.608110) | NFE Forward 50(49.9) | NFE Backward 50(49.1) | CNF Time 0.5000(0.5000)
Iter 1374 | Time 0.4244(0.4175) | Loss -0.784165(-0.620434) | NFE Forward 50(49.9) | NFE Backward 50(49.2) | CNF Time 0.5000(0.5000)
Iter 1375 | Time 0.4232(0.4179) | Loss -1.323673(-0.669661) | NFE For

Iter 1430 | Time 0.4229(0.4255) | Loss -0.370073(-0.685159) | NFE Forward 50(49.6) | NFE Backward 50(50.4) | CNF Time 0.5000(0.5000)
Iter 1431 | Time 0.4215(0.4253) | Loss -0.939960(-0.702995) | NFE Forward 50(49.6) | NFE Backward 50(50.4) | CNF Time 0.5000(0.5000)
Iter 1432 | Time 0.3939(0.4231) | Loss -1.078686(-0.729293) | NFE Forward 50(49.7) | NFE Backward 44(50.0) | CNF Time 0.5000(0.5000)
Iter 1433 | Time 0.4265(0.4233) | Loss -1.221396(-0.763740) | NFE Forward 50(49.7) | NFE Backward 50(50.0) | CNF Time 0.5000(0.5000)
Iter 1434 | Time 0.4251(0.4234) | Loss -1.253680(-0.798036) | NFE Forward 50(49.7) | NFE Backward 50(50.0) | CNF Time 0.5000(0.5000)
Iter 1435 | Time 0.4265(0.4236) | Loss -0.263111(-0.760591) | NFE Forward 50(49.7) | NFE Backward 50(50.0) | CNF Time 0.5000(0.5000)
Iter 1436 | Time 0.4271(0.4239) | Loss -0.079005(-0.712880) | NFE Forward 50(49.7) | NFE Backward 50(50.0) | CNF Time 0.5000(0.5000)
Iter 1437 | Time 0.4260(0.4240) | Loss -1.120102(-0.741386) | NFE For